In [ ]:
### Imports for the Project

In [3]:
import pandas as pd
import numpy as np
import copy
import torch
import tqdm
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import time

# ignore warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Neural Network Class Initialization

In [5]:
class Model(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, 82)
        self.fc2 = nn.Linear(82, 41)
        self.fc3 = nn.Linear(41, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
      x = self.relu(self.fc1(x))
      x = self.relu(self.fc2(x))
      x = self.sigmoid(self.fc3(x))
      return x

def model_train(model, X_train, y_train, X_val, y_val):
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 5   # number of epochs to run
    batch_size = 1 # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]

                # forward
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)

                # backward
                optimizer.zero_grad()
                loss.backward()

                # update weights
                optimizer.step()

                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )

        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())

    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    print("accuracy: ", best_acc)
    return best_acc

### Data Cleaning and Prep

In [6]:
# read data
data = pd.read_csv("seattle-weather.csv")

# Removing rows with missing values
data.dropna(inplace=True)

# Convert categorical variables
data = pd.get_dummies(data, columns=['temp_max', 'temp_min', 'wind'])
data = data.drop('date', axis=1)

# replace any sort of precip. with 1, sun/fog with 0
data['weather'].replace('rain', 1, inplace = True)
data['weather'].replace('drizzle', 1, inplace = True)
data['weather'].replace('fog', 0, inplace = True)
data['weather'].replace('sun', 0, inplace = True)
data['weather'].replace('snow', 1, inplace = True)

# replace precip. > 0 with 1
data['precipitation'] = data['precipitation'].apply(lambda x: 1 if x > 0 else x)

### Data Conversion

In [9]:
X = data.drop('precipitation', axis=1).astype(np.float64)
y = data['precipitation']

# converting data into tensors
X_tensors = torch.tensor(X.values, dtype=torch.float32)
y_tensors = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_tensors, y_tensors, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Train the Model and Output Accuracy

In [10]:
start = time.time()
input_size = X.shape[1]
print(input_size)
model = Model(input_size)
model_train(model, X_train, y_train, X_val, y_val)
end = time.time()
print("The model took", (end - start), "seconds to train")

202
accuracy:  0.965753436088562
The model took 9.349101543426514 seconds to train
